In [1]:
import pandas as pd
import json
import os
import deepsig
from IPython.display import display

2024-05-07 14:54:49.500104: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-07 14:54:49.500140: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-07 14:54:49.513289: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-07 14:54:49.539451: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-07 14:54:50.627499: W tensorflow/compiler/tf2

In [2]:
cols = ['dataset', 'method', 'fitness_rule', 'fitness', 'ACC', 'MCC', 'f1_score', 'avg_odds_diff', 'stat_par_diff', 'eq_opp_diff']

In [3]:
mlp_baseline_results = pd.read_csv('mlp_baseline_results.csv')
mlp_baseline_results.replace({'simple_mlp_initializer': 'Standard MLP (baseline)'}, inplace=True)

mlp_pearson_results = pd.read_csv('mlp_pearson_results.csv')
mlp_pearson_results.replace({'mlp_preg_initializer': 'Pearson regularized MLP'}, inplace=True)

mlp_spearman_results = pd.read_csv('mlp_spearman_results.csv')
mlp_spearman_results.replace({'mlp_sreg_initializer': 'Spearman regularized MLP'}, inplace=True)

mlp_results = pd.concat([mlp_baseline_results, mlp_pearson_results, mlp_spearman_results])

ftl_baseline_results = pd.read_csv('ftl_baseline_results.csv')
ftl_baseline_results.replace({'ftl_mlp_initializer': 'Standard FTL (baseline)'}, inplace=True)

ftl_pearson_results = pd.read_csv('ftl_pearson_results.csv')
ftl_pearson_results.replace({'ftl_mlp_preg_initializer': 'Pearson regularized FTL'}, inplace=True)

ftl_spearman_results = pd.read_csv('ftl_spearman_results.csv')
ftl_spearman_results.replace({'ftl_mlp_sreg_initializer': 'Spearman regularized FTL'}, inplace=True)

ftl_results = pd.concat([ftl_baseline_results, ftl_pearson_results, ftl_spearman_results])

In [4]:
for results in [mlp_results,ftl_results]:
    results.replace({'adult_dataset_reader': 'Adult Income', 'compas_dataset_reader': 'Compas Recidivism', 'german_dataset_reader': 'German Credit', 'bank_dataset_reader': 'Bank Marketing'}, inplace=True)
    results.rename(columns={'avg_odds_diff': 'Equalized Odds', 'stat_par_diff': 'Statistical Parity', 'eq_opp_diff': 'Equal Opportunity', 'MCC': 'Mathew Correlation', 'ACC': 'Accuracy'}, inplace=True)

In [5]:
fitness_rules_target_metrics = {
    'mcc_parity': {'performance': 'Mathew Correlation', 'fairness': 'Statistical Parity'},
    'mcc_opportunity': {'performance': 'Mathew Correlation', 'fairness': 'Equal Opportunity'},
    'mcc_odds': {'performance': 'Mathew Correlation', 'fairness': 'Equalized Odds'},
    'acc_parity': {'performance': 'Accuracy', 'fairness': 'Statistical Parity'},
    'acc_opportunity': {'performance': 'Accuracy', 'fairness': 'Equal Opportunity'},
    'acc_odds': {'performance': 'Accuracy', 'fairness': 'Equalized Odds'}
}

fitness_rules_target_metrics = {
    'mcc_parity': ('Mathew Correlation', 'Statistical Parity'),
    'mcc_opportunity': ('Mathew Correlation', 'Equal Opportunity'),
    'mcc_odds': ('Mathew Correlation', 'Equalized Odds'),
    'acc_parity': ('Accuracy', 'Statistical Parity'),
    'acc_opportunity': ('Accuracy', 'Equal Opportunity'),
    'acc_odds': ('Accuracy', 'Equalized Odds')
}
fitness_rules_abvr = {
    'mcc_parity': 'Max(MCC - Stat. Parity)',
    'mcc_opportunity': 'Max(MCC - Eq. Odds)',
    'mcc_odds': 'Max(MCC - Eq. Opp.)',
    'acc_parity': 'Max(Acc - Stat. Parity)',
    'acc_opportunity': 'Max(Acc - Eq. Odds)',
    'acc_odds': 'Max(Acc - Eq. Opp.)'
}

for results in [mlp_results,ftl_results]:
    results['Performance'] = 0
    results['Fairness'] = 0
    results['Fitness Rule'] = ''
    for fitness_rule, (performance_metric, fairness_metric) in fitness_rules_target_metrics.items():
        results.loc[results.fitness_rule == fitness_rule,'Performance'] = results.loc[results.fitness_rule == fitness_rule,performance_metric]
        results.loc[results.fitness_rule == fitness_rule,'Fairness'] = results.loc[results.fitness_rule == fitness_rule,fairness_metric]
        results.loc[results.fitness_rule == fitness_rule,'Fitness Rule Abvr'] = fitness_rules_abvr[fitness_rule]
        results.loc[results.fitness_rule == fitness_rule,'Fitness Rule'] = 'Max(%s - %s)' % fitness_rules_target_metrics[fitness_rule]

/tmp/ipykernel_117395/3607436483.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.58517004 0.58512823 0.5840247  0.5836371  0.57625737 0.5769683
 0.58075151 0.57811729 0.57147862 0.55847176 0.56402881 0.58191431
 0.58343253 0.57740222 0.51579307 0.30958237 0.37962947 0.55899189
 0.53558872 0.2598879  0.57717035 0.50086739 0.33275221 0.37929293
 0.47729109 0.29196869 0.29037704 0.52303609 0.27480633 0.38668781
 0.50626951 0.29960979 0.4042848  0.52119218 0.2936422  0.33522388
 0.49730721 0.25112662 0.35119067 0.510052   0.26843855 0.36081008
 0.52157495 0.30114722 0.42995147 0.48267704 0.29056691 0.23139881
 0.54223416 0.23653437 0.27041017 0.52121263 0.25759348 0.23912165
 0.5223892  0.27406652 0.22303564 0.54094974 0.27699558 0.30939251
 0.51731345 0.30339828 0.37825089 0.52113734 0.26323857 0.33963196
 0.5538225  0.27850228 0.30234984 0.51877126 0.29494101 0.2648939
 0.52742877 0.24492927 0.13407928 0.5353

In [6]:
display(mlp_results)

,dataset,method,fitness_rule,fitness,Accuracy,Mathew Correlation,Equalized Odds,Statistical Parity,Equal Opportunity,solution,Performance,Fairness,Fitness Rule,Fitness Rule Abvr
0,Adult Income,Standard MLP (baseline),mcc_parity,0.392537,0.851299,0.585170,0.113127,0.192633,0.142532,{'dropout': 0.16228954240968418},0.585170,0.192633,Max(Mathew Correlation - Statistical Parity),Max(MCC - Stat. Parity)
1,Adult Income,Standard MLP (baseline),mcc_odds,0.473935,0.851520,0.585500,0.111565,0.198915,0.132052,{'dropout': 0.1482543145261793},0.585500,0.111565,Max(Mathew Correlation - Equalized Odds),Max(MCC - Eq. Opp.)
2,Adult Income,Standard MLP (baseline),mcc_opportunity,0.460175,0.851410,0.585064,0.103235,0.191606,0.124889,{'dropout': 0.17861515209545933},0.585064,0.124889,Max(Mathew Correlation - Equal Opportunity),Max(MCC - Eq. Odds)
3,Adult Income,Standard MLP (baseline),acc_parity,0.661796,0.844887,0.565373,0.082386,0.183090,0.084174,{'dropout': 0.15655371267306142},0.844887,0.183090,Max(Accuracy - Statistical Parity),Max(Acc - Stat. Parity)
4,Adult Income,Standard MLP (baseline),acc_odds,0.766170,0.850857,0.577932,0.084687,0.180366,0.093891,{'dropout': 0.1698638434734938},0.850857,0.084687,Max(Accuracy - Equalized Odds),Max(Acc - Eq. Opp.)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,German Credit,Spearman regularized MLP,mcc_parity,0.335444,0.750000,0.395285,0.043544,0.059841,0.064362,"{'l2': 0.0001, 'dropout': 0.1297841491744075}",0.395285,0.059841,Max(Mathew Correlation - Statistical Parity),Max(MCC - Stat. Parity)
625,German Credit,Spearman regularized MLP,mcc_odds,0.360398,0.770000,0.464113,0.103715,0.095372,0.019024,"{'l2': 0.0001, 'dropout': 0.15909380030208975}",0.464113,0.103715,Max(Mathew Correlation - Equalized Odds),Max(MCC - Eq. Opp.)
626,German Credit,Spearman regularized MLP,mcc_opportunity,0.318623,0.725000,0.359842,0.055287,0.050948,0.041219,"{'l2': 0.01, 'dropout': 0.0931238477495196}",0.359842,0.041219,Max(Mathew Correlation - Equal Opportunity),Max(MCC - Eq. Odds)
627,German Credit,Spearman regularized MLP,acc_parity,0.624462,0.735000,0.326609,0.080675,0.110538,0.154024,"{'l2': 0.001, 'dropout': 0.16929347899646136}",0.735000,0.110538,Max(Accuracy - Statistical Parity),Max(Acc - Stat. Parity)


In [7]:
datasets = ['Adult Income', 'Bank Marketing', 'Compas Recidivism','German Credit']
datasets

['Adult Income', 'Bank Marketing', 'Compas Recidivism', 'German Credit']

In [8]:
fitness_rules = ['mcc_parity', 'mcc_opportunity', 'mcc_odds', 'acc_parity', 'acc_opportunity', 'acc_odds']
fitness_rules

['mcc_parity',
 'mcc_opportunity',
 'mcc_odds',
 'acc_parity',
 'acc_opportunity',
 'acc_odds']

In [9]:
ftl_methods = ['Standard FTL (baseline)', 'Pearson regularized FTL', 'Spearman regularized FTL']
mlp_methods = ['Standard MLP (baseline)', 'Pearson regularized MLP', 'Spearman regularized MLP']
significances = []

In [10]:
for path, methods, results in zip(['mlp_multi_aso_data_list.json', 'ftl_multi_aso_data_list.json'],
                                  [mlp_methods, ftl_methods],
                                  (mlp_results,ftl_results)):
    if os.path.exists(path):
        with open(path) as file:
            multi_aso_data_list = json.load(file)
    else:    
        multi_aso_data_list = []
        for d in datasets:
            multi_aso_data = []
            for f in fitness_rules:
                methods_results = []
                for m in methods:
                    r = results.loc[ (results['dataset'] == d) &
                                         (results['fitness_rule'] == f) &
                                         (results['method'] == m) ]\
                                .fitness.tolist()
                    if len(r) == 0:
                        r = [-1]
                    methods_results.append(r)
                min_eps = deepsig.multi_aso(methods_results, confidence_level=0.95)
                multi_aso_data_list.append({'fitness_rule': f, 'dataset': d, 'min_eps': min_eps.tolist()})
        with open(path, 'w') as file:
            json.dump(multi_aso_data_list, file)
    
    aso_df_resume = []
    for aso_result in sorted(multi_aso_data_list, key=lambda x: x['dataset']):
        fitness_rule = aso_result['fitness_rule']
        dataset = aso_result['dataset']
    
        aso_df = pd.DataFrame(aso_result['min_eps'], columns=methods)
        aso_df['method'] = methods
        aso_df['dataset'] = dataset
        aso_df['fitness_rule'] = fitness_rule
        aso_df_resume.append(aso_df)
    
    print('Significance Testing')
    significance = pd.concat(aso_df_resume)
    significance.replace(fitness_rules_abvr, inplace=True)
    significance = significance.set_index(['fitness_rule', 'dataset'])
    significance = significance.sort_values(by=['fitness_rule', 'dataset'], ascending=[False, True])
    display(significance)
    significances.append(significance)
    
    grouped_results = results\
        .groupby(['Fitness Rule Abvr', 'dataset', 'method'])\
        .agg({'fitness': ['mean', 'std', 'count'], 'Performance': ['mean', 'std'], 'Fairness': ['mean', 'std']})\
        .sort_values(by=['Fitness Rule Abvr', 'dataset', ('fitness','mean')], ascending=False)
    grouped_results['formatted_fitness'] = grouped_results.apply(lambda row: f"${row[('fitness', 'mean')]:.3f} (\pm{row[('fitness', 'std')]:.2f})$", axis=1)
    grouped_results['formatted_performance'] = grouped_results.apply(lambda row: f"${row[('Performance', 'mean')]:.3f} (\pm{row[('Performance', 'std')]:.2f})$", axis=1)
    grouped_results['formatted_fairness'] = grouped_results.apply(lambda row: f"${row[('Fairness', 'mean')]:.3f} (\pm{row[('Fairness', 'std')]:.2f})$", axis=1)
    grouped_results = grouped_results.sort_values(by=['Fitness Rule Abvr', 'dataset'])
    display(grouped_results)

Model comparisons: 100%|█████████▉| 2997/3000 [00:08<00:00, 367.88it/s]

Significance Testing


Standard MLP (baseline)  \
fitness_rule            dataset                                      
Max(MCC - Stat. Parity) Adult Income                      1.000000   
                        Adult Income                      0.715664   
                        Adult Income                      0.160297   
                        Bank Marketing                    1.000000   
                        Bank Marketing                    1.000000   
...                                                            ...   
Max(Acc - Eq. Odds)     Compas Recidivism                 0.366817   
                        Compas Recidivism                 0.618729   
                        German Credit                     1.000000   
                        German Credit                     1.000000   
                        German Credit                     0.577953   

                                           Pearson regularized MLP  \
fitness_rule            dataset                                      
Max(MCC - Stat. Parity) Adult Income                      1.000000   
                        Adult Income                      1.000000   
                        Adult Income                      0.309957   
                        Bank Marketing                    0.095254   
                        Bank Marketing                    1.000000   
...                                                            ...   
Max(Acc - Eq. Odds)     Compas Recidivism                 1.000000   
                        Compas Recidivism                 1.000000   
                        German Credit                     0.898449   
                        German Credit                     1.000000   
                        German Credit                     0.713411   

                                           Spearman regularized MLP  \
fitness_rule            dataset                                       
Max(MCC - Stat. Parity) Adult Income                       1.000000   
                        Adult Income                       1.000000   
                        Adult Income                       1.000000   
                        Bank Marketing                     0.079285   
                        Bank Marketing                     1.000000   
...                                                             ...   
Max(Acc - Eq. Odds)     Compas Recidivism                  0.847133   
                        Compas Recidivism                  1.000000   
                        German Credit                      1.000000   
                        German Credit                      1.000000   
                        German Credit                      1.000000   

                                                             method  
fitness_rule            dataset                                      
Max(MCC - Stat. Parity) Adult Income        Standard MLP (baseline)  
                        Adult Income        Pearson regularized MLP  
                        Adult Income       Spearman regularized MLP  
                        Bank Marketing      Standard MLP (baseline)  
                        Bank Marketing      Pearson regularized MLP  
...                                                             ...  
Max(Acc - Eq. Odds)     Compas Recidivism   Pearson regularized MLP  
                        Compas Recidivism  Spearman regularized MLP  
                        German Credit       Standard MLP (baseline)  
                        German Credit       Pearson regularized MLP  
                        German Credit      Spearman regularized MLP  

[72 rows x 4 columns]

fitness  \
                                                                        mean   
Fitness Rule Abvr       dataset           method                               
Max(Acc - Eq. Odds)     Adult Income      Spearman regularized MLP  0.768344   
                                          Standard MLP (baseline)   0.758290   
                                          Pearson regularized MLP   0.746905   
                        Bank Marketing    Spearman regularized MLP  0.838040   
                                          Pearson regularized MLP   0.814453   
...                                                                      ...   
Max(MCC - Stat. Parity) Compas Recidivism Standard MLP (baseline)   0.074028   
                                          Pearson regularized MLP   0.071768   
                        German Credit     Standard MLP (baseline)   0.265661   
                                          Spearman regularized MLP  0.261667   
                                          Pearson regularized MLP   0.261532   

                                                                              \
                                                                         std   
Fitness Rule Abvr       dataset           method                               
Max(Acc - Eq. Odds)     Adult Income      Spearman regularized MLP  0.044868   
                                          Standard MLP (baseline)   0.035161   
                                          Pearson regularized MLP   0.016776   
                        Bank Marketing    Spearman regularized MLP  0.051624   
                                          Pearson regularized MLP   0.068018   
...                                                                      ...   
Max(MCC - Stat. Parity) Compas Recidivism Standard MLP (baseline)   0.034694   
                                          Pearson regularized MLP   0.031838   
                        German Credit     Standard MLP (baseline)   0.099801   
                                          Spearman regularized MLP  0.086396   
                                          Pearson regularized MLP   0.082982   

                                                                          \
                                                                   count   
Fitness Rule Abvr       dataset           method                           
Max(Acc - Eq. Odds)     Adult Income      Spearman regularized MLP    15   
                                          Standard MLP (baseline)     15   
                                          Pearson regularized MLP     15   
                        Bank Marketing    Spearman regularized MLP    30   
                                          Pearson regularized MLP     30   
...                                                                  ...   
Max(MCC - Stat. Parity) Compas Recidivism Standard MLP (baseline)     30   
                                          Pearson regularized MLP     30   
                        German Credit     Standard MLP (baseline)     30   
                                          Spearman regularized MLP    30   
                                          Pearson regularized MLP     30   

                                                                   Performance  \
                                                                          mean   
Fitness Rule Abvr       dataset           method                                 
Max(Acc - Eq. Odds)     Adult Income      Spearman regularized MLP    0.847105   
                                          Standard MLP (baseline)     0.847864   
                                          Pearson regularized MLP     0.849022   
                        Bank Marketing    Spearman regularized MLP    0.900716   
                                          Pearson regularized MLP     0.901826   
...                                                                        ...   
Max(MCC - Stat. Parity) 

Model comparisons: 100%|█████████▉| 2997/3000 [00:06<00:00, 441.65it/s]

Significance Testing


Standard FTL (baseline)  \
fitness_rule            dataset                                      
Max(MCC - Stat. Parity) Adult Income                      1.000000   
                        Adult Income                      0.584574   
                        Adult Income                      1.000000   
                        Bank Marketing                    1.000000   
                        Bank Marketing                    0.338606   
...                                                            ...   
Max(Acc - Eq. Odds)     Compas Recidivism                 0.200783   
                        Compas Recidivism                 0.063887   
                        German Credit                     1.000000   
                        German Credit                     1.000000   
                        German Credit                     1.000000   

                                           Pearson regularized FTL  \
fitness_rule            dataset                                      
Max(MCC - Stat. Parity) Adult Income                      1.000000   
                        Adult Income                      1.000000   
                        Adult Income                      1.000000   
                        Bank Marketing                    1.000000   
                        Bank Marketing                    1.000000   
...                                                            ...   
Max(Acc - Eq. Odds)     Compas Recidivism                 1.000000   
                        Compas Recidivism                 0.763500   
                        German Credit                     0.843552   
                        German Credit                     1.000000   
                        German Credit                     1.000000   

                                           Spearman regularized FTL  \
fitness_rule            dataset                                       
Max(MCC - Stat. Parity) Adult Income                       0.415525   
                        Adult Income                       0.236786   
                        Adult Income                       1.000000   
                        Bank Marketing                     1.000000   
                        Bank Marketing                     1.000000   
...                                                             ...   
Max(Acc - Eq. Odds)     Compas Recidivism                  1.000000   
                        Compas Recidivism                  1.000000   
                        German Credit                      0.000000   
                        German Credit                      0.000000   
                        German Credit                      1.000000   

                                                             method  
fitness_rule            dataset                                      
Max(MCC - Stat. Parity) Adult Income        Standard FTL (baseline)  
                        Adult Income        Pearson regularized FTL  
                        Adult Income       Spearman regularized FTL  
                        Bank Marketing      Standard FTL (baseline)  
                        Bank Marketing      Pearson regularized FTL  
...                                                             ...  
Max(Acc - Eq. Odds)     Compas Recidivism   Pearson regularized FTL  
                        Compas Recidivism  Spearman regularized FTL  
                        German Credit       Standard FTL (baseline)  
                        German Credit       Pearson regularized FTL  
                        German Credit      Spearman regularized FTL  

[72 rows x 4 columns]

fitness  \
                                                                        mean   
Fitness Rule Abvr       dataset           method                               
Max(Acc - Eq. Odds)     Adult Income      Pearson regularized FTL   0.816948   
                                          Standard FTL (baseline)   0.815099   
                                          Spearman regularized FTL  0.796648   
                        Bank Marketing    Pearson regularized FTL   0.842059   
                                          Spearman regularized FTL  0.825748   
...                                                                      ...   
Max(MCC - Stat. Parity) Compas Recidivism Pearson regularized FTL   0.247669   
                                          Spearman regularized FTL  0.235185   
                        German Credit     Pearson regularized FTL   0.298430   
                                          Standard FTL (baseline)   0.256495   
                                          Spearman regularized FTL  0.183352   

                                                                              \
                                                                         std   
Fitness Rule Abvr       dataset           method                               
Max(Acc - Eq. Odds)     Adult Income      Pearson regularized FTL   0.023932   
                                          Standard FTL (baseline)   0.022059   
                                          Spearman regularized FTL  0.036947   
                        Bank Marketing    Pearson regularized FTL   0.037943   
                                          Spearman regularized FTL  0.051318   
...                                                                      ...   
Max(MCC - Stat. Parity) Compas Recidivism Pearson regularized FTL   0.045436   
                                          Spearman regularized FTL  0.047326   
                        German Credit     Pearson regularized FTL   0.061080   
                                          Standard FTL (baseline)   0.089117   
                                          Spearman regularized FTL  0.166189   

                                                                          \
                                                                   count   
Fitness Rule Abvr       dataset           method                           
Max(Acc - Eq. Odds)     Adult Income      Pearson regularized FTL     17   
                                          Standard FTL (baseline)     14   
                                          Spearman regularized FTL    13   
                        Bank Marketing    Pearson regularized FTL      5   
                                          Spearman regularized FTL     5   
...                                                                  ...   
Max(MCC - Stat. Parity) Compas Recidivism Pearson regularized FTL      5   
                                          Spearman regularized FTL     5   
                        German Credit     Pearson regularized FTL      4   
                                          Standard FTL (baseline)      4   
                                          Spearman regularized FTL     4   

                                                                   Performance  \
                                                                          mean   
Fitness Rule Abvr       dataset           method                                 
Max(Acc - Eq. Odds)     Adult Income      Pearson regularized FTL     0.844002   
                                          Standard FTL (baseline)     0.847485   
                                          Spearman regularized FTL    0.843398   
                        Bank Marketing    Pearson regularized FTL     0.882781   
                                          Spearman regularized FTL    0.897114   
...                                                                        ...   
Max(MCC - Stat. Parity) 

In [11]:
significances[0].sort_values(by=['fitness_rule', 'dataset'], ascending=[False, True])

Standard MLP (baseline)  \
fitness_rule            dataset                                      
Max(MCC - Stat. Parity) Adult Income                      1.000000   
                        Adult Income                      0.715664   
                        Adult Income                      0.160297   
                        Bank Marketing                    1.000000   
                        Bank Marketing                    1.000000   
...                                                            ...   
Max(Acc - Eq. Odds)     Compas Recidivism                 0.366817   
                        Compas Recidivism                 0.618729   
                        German Credit                     1.000000   
                        German Credit                     1.000000   
                        German Credit                     0.577953   

                                           Pearson regularized MLP  \
fitness_rule            dataset                                      
Max(MCC - Stat. Parity) Adult Income                      1.000000   
                        Adult Income                      1.000000   
                        Adult Income                      0.309957   
                        Bank Marketing                    0.095254   
                        Bank Marketing                    1.000000   
...                                                            ...   
Max(Acc - Eq. Odds)     Compas Recidivism                 1.000000   
                        Compas Recidivism                 1.000000   
                        German Credit                     0.898449   
                        German Credit                     1.000000   
                        German Credit                     0.713411   

                                           Spearman regularized MLP  \
fitness_rule            dataset                                       
Max(MCC - Stat. Parity) Adult Income                       1.000000   
                        Adult Income                       1.000000   
                        Adult Income                       1.000000   
                        Bank Marketing                     0.079285   
                        Bank Marketing                     1.000000   
...                                                             ...   
Max(Acc - Eq. Odds)     Compas Recidivism                  0.847133   
                        Compas Recidivism                  1.000000   
                        German Credit                      1.000000   
                        German Credit                      1.000000   
                        German Credit                      1.000000   

                                                             method  
fitness_rule            dataset                                      
Max(MCC - Stat. Parity) Adult Income        Standard MLP (baseline)  
                        Adult Income        Pearson regularized MLP  
                        Adult Income       Spearman regularized MLP  
                        Bank Marketing      Standard MLP (baseline)  
                        Bank Marketing      Pearson regularized MLP  
...                                                             ...  
Max(Acc - Eq. Odds)     Compas Recidivism   Pearson regularized MLP  
                        Compas Recidivism  Spearman regularized MLP  
                        German Credit       Standard MLP (baseline)  
                        German Credit       Pearson regularized MLP  
                        German Credit      Spearman regularized MLP  

[72 rows x 4 columns]

In [12]:
significances[1].sort_values(by=['fitness_rule', 'dataset'], ascending=[False, True])

Standard FTL (baseline)  \
fitness_rule            dataset                                      
Max(MCC - Stat. Parity) Adult Income                      1.000000   
                        Adult Income                      0.584574   
                        Adult Income                      1.000000   
                        Bank Marketing                    1.000000   
                        Bank Marketing                    0.338606   
...                                                            ...   
Max(Acc - Eq. Odds)     Compas Recidivism                 0.200783   
                        Compas Recidivism                 0.063887   
                        German Credit                     1.000000   
                        German Credit                     1.000000   
                        German Credit                     1.000000   

                                           Pearson regularized FTL  \
fitness_rule            dataset                                      
Max(MCC - Stat. Parity) Adult Income                      1.000000   
                        Adult Income                      1.000000   
                        Adult Income                      1.000000   
                        Bank Marketing                    1.000000   
                        Bank Marketing                    1.000000   
...                                                            ...   
Max(Acc - Eq. Odds)     Compas Recidivism                 1.000000   
                        Compas Recidivism                 0.763500   
                        German Credit                     0.843552   
                        German Credit                     1.000000   
                        German Credit                     1.000000   

                                           Spearman regularized FTL  \
fitness_rule            dataset                                       
Max(MCC - Stat. Parity) Adult Income                       0.415525   
                        Adult Income                       0.236786   
                        Adult Income                       1.000000   
                        Bank Marketing                     1.000000   
                        Bank Marketing                     1.000000   
...                                                             ...   
Max(Acc - Eq. Odds)     Compas Recidivism                  1.000000   
                        Compas Recidivism                  1.000000   
                        German Credit                      0.000000   
                        German Credit                      0.000000   
                        German Credit                      1.000000   

                                                             method  
fitness_rule            dataset                                      
Max(MCC - Stat. Parity) Adult Income        Standard FTL (baseline)  
                        Adult Income        Pearson regularized FTL  
                        Adult Income       Spearman regularized FTL  
                        Bank Marketing      Standard FTL (baseline)  
                        Bank Marketing      Pearson regularized FTL  
...                                                             ...  
Max(Acc - Eq. Odds)     Compas Recidivism   Pearson regularized FTL  
                        Compas Recidivism  Spearman regularized FTL  
                        German Credit       Standard FTL (baseline)  
                        German Credit       Pearson regularized FTL  
                        German Credit      Spearman regularized FTL  

[72 rows x 4 columns]